# The Mighty Thymio: Welcome 

In [1]:
import rospy

In [2]:
import rosnode
import rostopic
import rosmsg

In [77]:
import std_msgs.msg
import sensor_msgs.msg
import nav_msgs.msg

In [4]:
rospy.init_node('notebook')

In [30]:
def print_msg_definition(msg_type):
    print(rosmsg.get_msg_text(msg_type))

In [63]:
def print_first_message(topic, msg_type, number_of_messages=1, max_size=None):
    print("Let's print the first message[s]:\n")
    _ns = [number_of_messages]
    def callback(msg):
        _ns[0] -= 1
        if max_size:
            fmt = '%.{0}s ...'.format(max_size)
            print(fmt % msg)
        else:
             print(msg)
        if _ns[0] == 0:
            print('\nand then unsubscribe.')
            sub.unregister()
        
        
    sub = rospy.Subscriber(topic, msg_type, callback)


# [Raw] Aseba Messages

Aseba message are of two types: 
    - asebaros_msgs/AsebaEvent 
    - asebaros_msgs/AsebaAnonymousEvent

In [13]:
for msg_type in rosmsg.list_msgs('asebaros_msgs'):
    print('\n------- %s -------' % msg_type)
    print('Format:\n%s' % rosmsg.get_msg_text(msg_type))
    print('Topics:\n%s' % '\n'.join(rostopic.find_by_type(msg_type)))


------- asebaros_msgs/AsebaAnonymousEvent -------
Format:
time stamp
uint16 source
uint16 type
int16[] data

Topics:
/aseba/anonymous_events

------- asebaros_msgs/AsebaEvent -------
Format:
time stamp
uint16 source
int16[] data

Topics:
/aseba/events/set_speed
/aseba/events/ground_ambient
/aseba/events/buttons
/aseba/events/set_led_bottom_left
/aseba/events/ground_reflected
/aseba/events/play_sound
/aseba/events/sound
/aseba/events/button_left
/aseba/events/tap
/aseba/events/button_right
/aseba/events/temperature
/aseba/events/play_system_sound
/aseba/events/button_forward
/aseba/events/remote
/aseba/events/button_backward
/aseba/events/shutdown
/aseba/events/proximity
/aseba/events/set_comm
/aseba/events/comm
/aseba/events/ground
/aseba/events/set_led_top
/aseba/events/set_led_bottom_right
/aseba/events/accelerometer
/aseba/events/set_led_gesture
/aseba/events/set_led
/aseba/events/set_sound_threshold
/aseba/events/button_center
/aseba/events/odometry


In [14]:
import asebaros_msgs.msg

print_first_message('/aseba/events/proximity', asebaros_msgs.msg.AsebaEvent)

Let's print the first message[s]:

stamp: 
  secs: 1519316764
  nsecs: 558772005
source: 1
data: [0, 0, 0, 0, 0, 0, 0]

and then unsubscribe.


# Sensors

## Proximity Sensors

Data from proximity sensors are of type sensor_msgs/Range

In [15]:
print(rosmsg.get_msg_text("sensor_msgs/Range"))

uint8 ULTRASOUND=0
uint8 INFRARED=1
std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
uint8 radiation_type
float32 field_of_view
float32 min_range
float32 max_range
float32 range



In [16]:
rostopic.find_by_type('sensor_msgs/Range')

['/proximity/center_right',
 '/ground/right',
 '/ground/left',
 '/proximity/center',
 '/proximity/rear_left',
 '/proximity/left',
 '/proximity/rear_right',
 '/proximity/right',
 '/proximity/center_left']

There are 5 proximity sensor on the front
     - `/proximity/left`,
     - `/proximity/center_left`
     - `/proximity/center`
     - `/proximity/center_right`
     - `/proximity/right`)
2 on the back
    - `/proximity/rear_left`
    - `/proximity/rear_right`
and  2 pointing down
    - `/ground/right`
    - `/ground/right`

In [17]:
import sensor_msgs.msg

print_first_message('/proximity/center', sensor_msgs.msg.Range)

Let's print the first message[s]:

header: 
  seq: 8
  stamp: 
    secs: 1519316776
    nsecs: 233592033
  frame_id: "proximity_center_link"
radiation_type: 1
field_of_view: 0.300000011921
min_range: 0.0215000007302
max_range: 0.140000000596
range: inf

and then unsubscribe.


## Proximity Sensors
Data from the microphone is of type 

## Buttons

The buttons state is periodically published as an aggregate `sensor_msgs/Joy` message 

In [18]:
print(rostopic.get_topic_type('/buttons')[0])

sensor_msgs/Joy


In [19]:
print(rosmsg.get_msg_text('sensor_msgs/Joy'))

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
float32[] axes
int32[] buttons



In [20]:
print_first_message('buttons', sensor_msgs.msg.Joy)

Let's print the first message[s]:

header: 
  seq: 7
  stamp: 
    secs: 1519316788
    nsecs: 919080972
  frame_id: ''
axes: []
buttons: [0, 0, 0, 0, 0]

and then unsubscribe.


When the state of individual buttons is changed, an `std_msgs/Bool` is published:

In [21]:
rospy.get_published_topics('/buttons')

[['/buttons/backward', 'std_msgs/Bool'],
 ['/buttons/right', 'std_msgs/Bool'],
 ['/buttons/left', 'std_msgs/Bool'],
 ['/buttons/forward', 'std_msgs/Bool'],
 ['/buttons/center', 'std_msgs/Bool']]

In [22]:
print_first_message('/buttons/forward', std_msgs.msg.Bool, number_of_messages=2)

Let's print the first message[s]:

data: True
data: False

and then unsubscribe.


# Camera

The camera driver publishes two types of topics.

`/camera/camera_info` contains the informaion about the image size and the lens distortion.

In [28]:
print(rostopic.get_topic_type('/camera/camera_info')[0])

sensor_msgs/CameraInfo


In [31]:
print_msg_definition('sensor_msgs/CameraInfo')

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
uint32 height
uint32 width
string distortion_model
float64[] D
float64[9] K
float64[9] R
float64[12] P
uint32 binning_x
uint32 binning_y
sensor_msgs/RegionOfInterest roi
  uint32 x_offset
  uint32 y_offset
  uint32 height
  uint32 width
  bool do_rectify



The other topics provide the (optionally compressed) camera images
    `camera/image_raw[/compressed|/theora]`

In [35]:
for compr in ['', '/compressed', '/theora']:
    topic = '/camera/image_raw%s' % compr
    msg_type = rostopic.get_topic_type(topic)[0]
    print('%s: %s' % (topic, msg_type))

/camera/image_raw: sensor_msgs/Image
/camera/image_raw/compressed: sensor_msgs/CompressedImage
/camera/image_raw/theora: theora_image_transport/Packet


In [36]:
print_msg_definition('sensor_msgs/Image')

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
uint32 height
uint32 width
string encoding
uint8 is_bigendian
uint32 step
uint8[] data



In [37]:
print_msg_definition('sensor_msgs/CompressedImage')

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
string format
uint8[] data



In [64]:
print_first_message('/camera/image_raw', sensor_msgs.msg.Image, max_size=1000)

Let's print the first message[s]:

header: 
  seq: 1014
  stamp: 
    secs: 1519317736
    nsecs: 850014857
  frame_id: "camera_link"
height: 480
width: 640
encoding: "rgb8"
is_bigendian: 0
step: 1920
data: [74, 54, 0, 73, 53, 0, 68, 45, 2, 66, 43, 0, 70, 44, 20, 71, 45, 21, 69, 44, 24, 69, 44, 24, 70, 44, 18, 70, 44, 18, 68, 44, 10, 62, 38, 4, 60, 39, 0, 60, 39, 0, 60, 42, 0, 58, 40, 0, 62, 43, 3, 47, 28, 0, 30, 10, 0, 43, 23, 0, 41, 23, 0, 41, 23, 0, 37, 24, 0, 35, 22, 0, 30, 22, 0, 25, 17, 0, 16, 13, 0, 9, 6, 0, 0, 0, 2, 1, 1, 3, 0, 1, 7, 0, 1, 7, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1, 0, 9, 1,

## (inter-robot) IR  communication
Robots can send to neighbors small chanks of 10 bits encoded as a `std_msgs/Int16`

In [69]:
print(rostopic.get_topic_type('/comm/receive')[0])

std_msgs/Int16


In [68]:
# print_first_message('/camera/image_raw', std_msgs.msg.Int16)

Let's print the first message[s]:



## IMU

In [72]:
print_msg_definition('sensor_msgs/Imu')

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
geometry_msgs/Quaternion orientation
  float64 x
  float64 y
  float64 z
  float64 w
float64[9] orientation_covariance
geometry_msgs/Vector3 angular_velocity
  float64 x
  float64 y
  float64 z
float64[9] angular_velocity_covariance
geometry_msgs/Vector3 linear_acceleration
  float64 x
  float64 y
  float64 z
float64[9] linear_acceleration_covariance



In [71]:
print_first_message('/imu',sensor_msgs.msg.Imu)

Let's print the first message[s]:

header: 
  seq: 1
  stamp: 
    secs: 1519318057
    nsecs: 188226938
  frame_id: "base_link"
orientation: 
  x: 0.0
  y: 0.0
  z: 0.0
  w: 0.0
orientation_covariance: [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
angular_velocity: 
  x: 0.0
  y: 0.0
  z: 0.0
angular_velocity_covariance: [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
linear_acceleration: 
  x: 0.853043478261
  y: 0.0
  z: 9.38347826087
linear_acceleration_covariance: [0.07, 0.0, 0.0, 0.0, 0.07, 0.0, 0.0, 0.0, 0.07]

and then unsubscribe.


In [ ]:
## TODO: joint_states

In [74]:
# print_first_message('/joint_states', sensor_msgs.msg.JointState)

Let's print the first message[s]:



# Odometry

In [80]:
print_msg_definition('nav_msgs/Odometry')

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
string child_frame_id
geometry_msgs/PoseWithCovariance pose
  geometry_msgs/Pose pose
    geometry_msgs/Point position
      float64 x
      float64 y
      float64 z
    geometry_msgs/Quaternion orientation
      float64 x
      float64 y
      float64 z
      float64 w
  float64[36] covariance
geometry_msgs/TwistWithCovariance twist
  geometry_msgs/Twist twist
    geometry_msgs/Vector3 linear
      float64 x
      float64 y
      float64 z
    geometry_msgs/Vector3 angular
      float64 x
      float64 y
      float64 z
  float64[36] covariance



In [79]:
print_first_message('/odom', nav_msgs.msg.Odometry)

Let's print the first message[s]:

header: 
  seq: 1
  stamp: 
    secs: 1519318234
    nsecs: 568774938
  frame_id: "odom"
child_frame_id: "base_link"
pose: 
  pose: 
    position: 
      x: -0.473507056934
      y: -0.739992350704
      z: 0.0
    orientation: 
      x: 0.0
      y: 0.0
      z: 0.404883710728
      w: -0.914368186666
  covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
twist: 
  twist: 
    linear: 
      x: 0.000511945392491
      y: 0.0
      z: 0.0
    angular: 
      x: 0.0
      y: 0.0
      z: 0.0107777977367
  covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

and then unsubscribe.


In [ ]:
## TODO remote

## TODO sound

In [83]:
print_first_message('/sound', std_msgs.msg.Float32)

Let's print the first message[s]:

data: 0.145098045468

and then unsubscribe.


In [ ]:
## TODO Tap

In [84]:
print_first_message('/tap', std_msgs.msg.Empty)

Let's print the first message[s]:



and then unsubscribe.


In [ ]:
#TODO temp

In [86]:
print_first_message('/temperature', sensor_msgs.msg.Temperature)

Let's print the first message[s]:

header: 
  seq: 4
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: "base_link"
temperature: 31.1
variance: 0.01

and then unsubscribe.


In [ ]:
# TODO

# /ups/ac
# /ups/battery

In [87]:
print_first_message('/ups/ac', std_msgs.msg.Bool)

Let's print the first message[s]:

data: True

and then unsubscribe.


In [88]:
print_first_message('/ups/battery', std_msgs.msg.Bool)

Let's print the first message[s]:

data: True

and then unsubscribe.
